<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/11_panel_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11.　企業財務データのパネルデータ分析

　本例では、2010年から2017年のクアラルンプール（マレーシア）上場企業の財務データを用い、企業の資本構成の決定理論が実証的に整合するかについての分析例を示します。パネルデータを分析するパッケージとして、linearmodels 4.27を用います。まず最初のセルを用いてこのパッケージをインストールします。

In [ ]:
pip install linearmodels

　パッケージのインストール後、ライブラリのインポートを行います。パネルデータ分析用のライブラリは変量効果モデル、固定効果モデル、プーリングOLS、それぞれのライブラリをインポートします。3つの推計結果の一覧表を作成するためのライブラリも併せて読み込みます。

In [2]:
#[1]ライブラリの読み込み
import pandas as pd
from linearmodels import RandomEffects #変量効果モデルのライブラリ
from linearmodels import PanelOLS #固定効果モデルのライブラリ
from linearmodels import PooledOLS #プーリングOLSモデルのライブラリ
from linearmodels.panel import compare #一覧表作成のライブラリ
import statsmodels.api as sm

　本例では、クアラルンプール上場企業の財務データをGitHubから取得します。取得したデータはdfというデータフレーム名で、pandasデータフレームに保存します。ちなみに読み込んだ変数名の定義は次の通りです。

year：決算年,
firm:企業番号,
DER：負債・資本（時価）比率	各企業の各年度の負債／時価資本,
ROA：総資産利益率		各企業の各年度の純利益／総資産,
Ebit_Asset：EBIT対総資産比率	各企業の各年度の利払・税引前利益／総資産,
Asset：総資産残高規模		各企業の各年度の総資産残高（百万USD）の自然対数値,
Tangible_sales：固定資産対売上高比　各企業の各年度の固定資産の売上高比,
ICR：金利負担余力		各企業の各年度のキャッシュフロー対支払金利比


In [3]:
#[2]標本データの取得
url = "https://github.com/nagamamo/data/blob/main/11_malaysian_firm_data.csv?raw=true"#Git-hubからCSVデータの入手
df = pd.read_csv(url)#データフレームの作成

  [3]ではpandasデータフレームのデータをパネルデータとして用いるための前処理を行います。具体的には、時間(year）と企業（firm）の２つのデータ定義をデータフレームに含めることで、後述のパネルデータ推計が行える処理をします。

In [4]:
#[3]データ前処理
year=pd.Categorical(df.year) #時間カテゴリー変数の作成と定義
df['year']=year
df=df.set_index(['year','firm']) #パネルデータ定義
df.head()

　変量効果モデルの推計を行います。説明変数にはROA、ASSET、Tabgible_salesを選択し、被説明変数にはDERを選択します。mod=RandomEffects()の（）内に被説明変数と、一行目で選択した説明変数のグループ名を記述します。

In [ ]:
#[4]変量効果モデルの推計
exog_vars=['ROA','ASSET','Tangible_sales'] #説明変数の選択
exog=sm.add_constant(df[exog_vars]) #定数項の指示
mod=RandomEffects(df.DER,exog) #実証モデルの指示
re_res=mod.fit()
re_res #実証結果の表示

　同様の手順で、固定効果モデル、プーリングOLSモデルを推計します。重要な点は、３つの推計のうち、固定効果モデルのみ、time_effects=Trueと記述することで、時間効果を推計モデルに加えている点です。また、mod=PanelOLS()では、（）内にentity_effects=Trueと定義することで、固定効果の存在を指示しています。

In [ ]:
#[5]固定効果モデルの推計
exog_vars=['ROA','ASSET','Tangible_sales']
exog=sm.add_constant(df[exog_vars])
mod=PanelOLS(df.DER,exog,entity_effects=True,time_effects=True) #固定効果と時間効果を=True
fe_res=mod.fit()
fe_res

In [ ]:
#[6]プーリングOLSモデル
exog_vars=['ROA','ASSET','Tangible_sales']
exog=sm.add_constant(df[exog_vars])
mod=PooledOLS(df.DER,exog)
pooled_res=mod.fit()
pooled_res

最後に３つの実証モデル推計の結果一覧表を表示します。

In [9]:
#[7]結果一覧表の表示
compare({"RE":re_res,"FE":fe_res,"Pooled":pooled_res})

,RE,FE,Pooled
Dep. Variable,DER,DER,DER
Estimator,RandomEffects,PanelOLS,PooledOLS
No. Observations,6591,6591,6591
Cov. Est.,Unadjusted,Unadjusted,Unadjusted
R-squared,0.0218,0.0054,0.0217
R-Squared (Within),0.0219,0.0209,0.0219
R-Squared (Between),-0.0606,-0.0405,-0.0604
R-Squared (Overall),0.0217,0.0208,0.0217
F-statistic,48.846,10.312,48.786
P-value (F-stat),0.0000,0.0000,0.0000
